In [1]:
import pandas as pd
import numpy as np


In [2]:
csv_data = pd.read_csv(r'CleanData.csv');

In [3]:
trials = range(1,41)
bpm = [80,100,120]
gestureSize = ["S","M","L"]

In [4]:
csv_data.DistanceCoveredSoFar = csv_data.DistanceCoveredSoFar.astype(float)
csv_data.VelocityMagnitude = csv_data.VelocityMagnitude.astype(float)
csv_data.AngleToBP1 = csv_data.AngleToBP1.astype(float)
csv_data.BPM = csv_data.BPM.astype(int)
csv_data.Position_Y = csv_data.Position_Y.astype(float)

In [5]:
newOutputTable = pd.DataFrame({'Trial': [], 'BPM': [], 'GestureSize': [], 'MaxAngleRegionOne': [], 'MaxAngleRegionTwo': []})

In [6]:
for g in gestureSize:
    for b in bpm:
        for i in trials:
            sliceByTrial = csv_data.loc[csv_data.Trial == i]
            sliceByGesture = sliceByTrial.loc[sliceByTrial.GestureSize == g]
            sliceByBPM = sliceByGesture.loc[sliceByGesture.BPM == b]
            
            
            # =======================================
            # ============ REGION 1 =================
            # =======================================
            
            globalHighestPointIndex = sliceByBPM.Position_Y.idxmax() #global highest Position_Y
            localHighestPointIndex = globalHighestPointIndex - sliceByBPM.index[0] #index of local highest point
            
            sliceToGetPositiveYVelocity = sliceByBPM.loc[sliceByBPM.Velocity_Y > 0]
            globalIndexFirstPositiveYVelocity = sliceToGetPositiveYVelocity.index[0]
            localIndexFirstPositiveYVelocity = globalIndexFirstPositiveYVelocity - sliceByBPM.index[0]

            RegionOne = sliceByBPM[localIndexFirstPositiveYVelocity:localHighestPointIndex+1]
            
            # =======================================
            # ============ REGION 2 =================
            # =======================================
            
            smallestYPoint_RegionOneIndex = RegionOne.Position_Y.idxmin()
            
            # We get the highest Y position with our globalHighestPointIndex
            y_positionHighestPoint = sliceByBPM.Position_Y[globalHighestPointIndex]
            # And our lowest y position with our smallest y point in region 1
            y_positionLowestPoint  = sliceByBPM.Position_Y[smallestYPoint_RegionOneIndex]
            
            # We get the absolute y distance between the lowest and highest points
            y_distance = np.absolute(y_positionHighestPoint - y_positionLowestPoint)
            # Then we take half of that distance
            y_distanceWithinRegion2 = y_distance/2
            
            # Now let's get everything NOT in Region 1
            portionAfterRegionOne = sliceByBPM[localHighestPointIndex+1:]
            
            # From everything NOT in Region 1 we grab every point that is within y_distanceWithinRegion2 of the highest Y position in Region 1
            # And like that we have Region Two 
            RegionTwo = portionAfterRegionOne[np.absolute(portionAfterRegionOne.Position_Y - y_positionHighestPoint) < y_distanceWithinRegion2]
            
            
               
            # =======================================
            # ============== Max Angle ==============
            # =======================================
            
            
            # maxAngleSoFar = 0
            # for angle in slicedLocalData["AngleToBP1"]:
               # if (angle > maxAngleSoFar):
                 #   maxAngleSoFar = angle
            maxAngleRegionOne = max(RegionOne["AngleToBP1"])
            maxAngleRegionTwo = max(RegionTwo["AngleToBP1"])
            newOutputTable = newOutputTable.append({'Trial': i,'BPM':b,'GestureSize':g,'MaxAngleRegionOne':maxAngleRegionOne, 'MaxAngleRegionTwo': maxAngleRegionTwo}, ignore_index = True)

In [7]:
newOutputTable

,BPM,GestureSize,MaxAngleRegionOne,MaxAngleRegionTwo,Trial
0,80.0,S,67.87199,69.45491,1.0
1,80.0,S,69.85532,70.92043,2.0
2,80.0,S,68.08369,67.91116,3.0
3,80.0,S,75.15100,77.19430,4.0
4,80.0,S,71.39948,71.43022,5.0
5,80.0,S,68.97768,68.98109,6.0
6,80.0,S,71.99378,72.95826,7.0
7,80.0,S,69.70382,70.11354,8.0
8,80.0,S,71.18315,71.30025,9.0
9,80.0,S,70.62938,71.39008,10.0


In [8]:
newOutputTable.to_csv(r'MaxAnglePerTrial.csv')